In [ ]:
import os
import tarfile
import urllib
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms
from PIL import Image
import ssl

# create SSL context
ssl._create_default_https_context = ssl._create_unverified_context

# Download the CIFAR-10 dataset
url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
file_name = "cifar-10-python.tar.gz"

if not os.path.exists(file_name):
    urllib.request.urlretrieve(url, file_name)
    print("Dataset downloaded")

# Extract the dataset
if not os.path.exists("cifar-10-batches-py"):
    with tarfile.open(file_name, "r:gz") as tar:
        tar.extractall()
    print("Dataset extracted")


In [ ]:
from tqdm import tqdm
from utils import CIFAR10Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms

if torch.cuda.is_available():
    device = 'cuda'
    print("Using cuda")
else:
    device = 'cpu'
    print("Using cpu")
    
# Define transforms for data normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Create train and validation datasets
trainset = CIFAR10Dataset(root_dir="cifar-10-batches-py", transform=transform, train=True)
valset = CIFAR10Dataset(root_dir="cifar-10-batches-py", transform=transform, train=False)

# Create train and validation dataloaders
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
valloader = DataLoader(valset, batch_size=64, shuffle=False, num_workers=2)

# Define the network architecture


class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()

        # Convolutional layers
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=2),  # Adjusted kernel size and padding
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Adaptive average pooling
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))  # Adjusted output size

        # Fully connected layers
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),

            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),

            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


net = AlexNet()
net = net.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 100
best_accuracy = 0.0
patience = 5  # Number of epochs to wait for improvement
counter = 0  # Counter for early stopping
for epoch in range(num_epochs):
    running_loss = 0.0
    with tqdm(total=len(trainloader), desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as pbar:
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 200 == 199:
                pbar.set_postfix({'Loss': running_loss/200})
                running_loss = 0.0
            pbar.update(1)

    # Validation loop
    correct = 0
    total = 0
    with torch.no_grad():
        for data in valloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Validation accuracy: {accuracy:.2f}%')

    # Check if the current accuracy is better than the previous best accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        counter = 0  # Reset the counter
    else:
        counter += 1

    # Check if early stopping criteria is met
    if counter >= patience:
        print('Early stopping! Validation accuracy stopped improving.')
        break

print('Training finished')

# Evaluate the network on the test set
correct = 0
total = 0
with torch.no_grad():
    with tqdm(total=len(valloader), desc='Testing', unit='batch') as pbar:
        for data in valloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            pbar.update(1)

accuracy = 100 * correct / total
print(f'Accuracy of the network on the {len(valset)} validation images: {accuracy:.2f}%')


In [ ]:
import torchvision.models as models

model = models.resnet18()


In [1]:
from tqdm import tqdm
from utils import CIFAR10Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms

device = 'cpu'
    
# Define transforms for data normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Create train and validation datasets
trainset = CIFAR10Dataset(root_dir="cifar-10-batches-py", transform=transform, train=True)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

for i, data in enumerate(trainloader, 0):
    inputs, labels = data[0].to(device), data[1].to(device)

In [9]:
import torch.nn as nn
import torchvision

# modifed Alexnet
class AlexNet2(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet2, self).__init__()
        self.alexnet = torchvision.models.alexnet(pretrained=False)
        self.alexnet.classifier[-1] = nn.Linear(in_features=4096, out_features=10, bias=True)

    def forward(self, x):
        x = self.alexnet(x)
        return x
    
net = AlexNet2()
# net.alexnet.classifier[-1] = nn.Linear(in_features=4096, out_features=10, bias=True)
net

AlexNet2(
  (alexnet): AlexNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
    (classifier): Sequential(
      (0): Dropout(p=0.5, inplace=False)
      